# TP2
## 1 Algorithme $A*$

Le graphe dirigé de l'énoncé représente le graphe d’état et les coûts de transitions de votre problème.
On donne $S_I = S$ et $S_G = G$.

### 1.1 Recherche aveugle
Dans un tableau de 3 colonnes renseignez les noeuds de l’arbre, la valeur de la fonction d’évaluation et leur parent dans l’ordre dans lequels ils seront développés dans une
recherche aveugle de votre choix (vous pouvez dessiner l’arbre de recherche).

Pour résoudre ce problème, je vais utiliser un algorithme Best First Search. Pour choisir les noeuds, on va utiliser un ordre alphabétique inversé. Donc par exemple, si on a A, B et C comme noeuds voisins,
on va d'abord explorer la branche passant par C, puis par B, puis par A... Dans notre cas, on a A et F, donc on va commencer par visiter F. Ce qui nous donne le tableau suivant, avec les noeuds visités dans
l'ordre alphabétique inversé:

| Node | Parent | Cost |
|:----:|:------:|:----:|
| S | None | 0 |
| F | S | 7 |
| H | F | 9 |
| G | F | 12 |

### 1.2 Recherche heuristique
Le tableau suivant propose une heuristique h telle que h(s) estime le coût du chemin jusqu’à
l’état final SG dans le graphe

| Etat | S | A | C | D | F | G | H |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| h | 10 | 5 | 4 | 3 | 4 | 0 | 2 |

1. Cette fonction heuristique est-elle admissible ? Pourquoi ?

    Une heuristique est admissible si elle sous-estime le coût du chemin vers la solution. Donc si $h^*(v)$ est le coût réel du chemin allant jusqu'au noeud $v$, alors si $h$ est admissible,
    on aura:
    $$0 \leq h(v) \leq h^*(v)$$
    Dans notre cas, on a $0 \leq h(G) = 0 \leq h^*(G) = 11 $ (j'ai calculé de tête le coût minimal de la solution avec la pondération du graphe...). Donc notre heuristique est admissible. Et elle est de toute façon admissible car $h(G) = 0$ et que $h^*(G) \geq 0$ donc $h^*(G) \geq h(G) \geq 0$. Donc même sans calculer de tête $h^*(G)$, comme le coût est supposé toujours positif, on a que notre heuristique est admissible.

2. Cette fonction heuristique est-elle consistante ? Pourquoi ?

    Une heuristique est dite consistante si elle maintient cet ordre:

    S'il existe une transition allant de $s$ à $s'$ de coût $c(s, s')$, alors
    $$h(s) \leq h(s') + c(s, s')$$

    Dans notre cas, on a:
    $h(A) = 5 \leq h(S) + c(F, A) = 4 + 8 = 12$ Ok

    $h(F) = 4 \leq h(A) + c(A, F) = 5 + 8 = 13$ Ok

    $h(G) = 0 \leq h(F) + c(F, G) = 4 + 5 = 9$ Ok // À considérer ???

    $h(G) = 0 \leq h(C) + c(C, G) = 4 + 5 = 9$ Ok // Idem
    
    $h(G) = 0 \leq h(D) + c(D, G) = 3 + 3 = 6$ Ok // Idem

    Notre fonction est donc consistante


3. Quelles sont les fonctions d'évaluation utilisées par la méthode _greedy best first search_ et l'algorithme $A^*$ ?

    La fonction d'évaluation utilisée par la méthode _greedy best first search_ est donnée par:
    $$f(v) = h(v)$$
    La fonction d'évaluation utilisée par l'algorithme $A^*$ est donnée par:
    $$f(v) = g(v) + h(v)$$
    La fonction $g(v)$ est la prise en compte de la connaissance actuelle de notre système pour mieux se diriger vers la solution optimale.

In [8]:
# This function return a list of neighbourg from a node given, sorted in decrease cost
# because we takes the elements by the end of the list...
def list_neighbourg(node: dict[str, int], graph: dict[str, list[tuple[str, int]]]):
    result = []
    for i in graph[node[0]]:
        # Here we store values with just the evaluating function equal to h(G)
        result.append((i[0], h[i[0]]))
    result.sort(key=lambda item: item[1], reverse=True)
    return result

# This function is greedy best first search
def Greedy_BFS(start: str, goal: str, graph, h: dict[str, int]):
    visited = []
    stack = []
    parents = {}
    node = ""
    stack.append((start, h[start]))
    # We visit the graph
    while (len(stack) != 0):
        tmp = stack.pop()
        node = (tmp[0], tmp[1] - h[tmp[0]])
        visited.append(node[0])
        if (node[0] == goal):
            break
        neighbourg = list_neighbourg(node, graph)
        for i in neighbourg:
            if i[0] not in visited:
                tmp = (i[0], i[1] - h[i[0]])
                # Parents is used for backtracking
                parents[tmp] = node
                stack.append(i)
    # If we find a path, we backtrack this path
    if node[0] == goal:
        path = []
        path.append(node[0])
        parent = node
        while parent in parents.keys():
            parent = parents[parent]
            path.append(parent[0])
        return path[::-1]
    return []

h = {"S":10, "A":5, "C":4, "D":3, "F":4, "G":0, "H":2}

graph = {"S":[("F", 7), ("A", 2)], "F":[("H", 2), ("G", 5), ("A", 8)], "H":[], "G":[], "D": [("G", 3)], "C": [("D", 3), ("G", 5)], "A": [("F", 8), ("C", 4)]}

print(Greedy_BFS("S", "G", graph, h))


['S', 'F', 'G']


5. Comment transformer cette implémentation en l’algorithme $A^∗$?
    
    Pour transformer cette implémentation en l'algorithme $A^*$, il faut ajouter à notre heuristique $h$ une fonction $g$ qui prend en compte l'état du système afin de diriger l'algorithme vers la solution optimale.
    Dans notre cas, on peut définir $g(v)$ = coût du chemin déjà parcouru.
    Ainsi, notre fonction d'évaluation nous donnera:
    $$f(v) = g(v) + h(v)$$
    avec $h(v)$ la fonction définie précédement.

In [9]:
# This function return a list of neighbourgs from a node given
def list_neighbourg(node: dict[str, int], graph: dict[str, list[tuple[str, int]]]):
    result = []
    for i in graph[node[0]]:
        g = i[1] + node[1]
        # Here we have our evaluating function equal to g(G) + h(G) for a node G
        result.append((i[0], g + h[i[0]]))
    return result

# This is the algorithm A*
def A_Star(start: str, goal: str, graph, h: dict[str, int]):
    visited = []
    stack = []
    parents = {}
    node = ""
    stack.append((start, h[start]))
    # We iterate over our graph
    while (len(stack) != 0):
        tmp = stack.pop()
        node = (tmp[0], tmp[1] - h[tmp[0]])
        visited.append(node[0])
        if (node[0] == goal):
            break
        neighbourg = list_neighbourg(node, graph)
        for i in neighbourg:
            if i[0] not in visited:
                # Here we store the node without the heuristic function h for easily backtrack
                # the solution later...
                tmp = (i[0], i[1] - h[i[0]])
                parents[tmp] = node
                stack.append(i)
        # We sort all nodes in decrease cost because we choose the last node, so the node
        # with the smallest cost
        stack.sort(key=lambda item: item[1], reverse=True)
    # We backtrack the solution
    if node[0] == goal:
        path = []
        path.append(node)
        parent = node
        while parent in parents.keys():
            parent = parents[parent]
            path.append(parent)
        return path[::-1]

    return []

h = {"S":10, "A":5, "C":4, "D":3, "F":4, "G":0, "H":2}

graph = {"S":[("F", 7), ("A", 2)], "F":[("H", 2), ("G", 5), ("A", 8)], "H":[], "G":[], "D": [("G", 3)], "C": [("D", 3), ("G", 5)], "A": [("F", 8), ("C", 4)]}

print(A_Star("S", "G", graph, h))
        

[('S', 0), ('A', 2), ('C', 6), ('G', 11)]


6. Pour chacune de ces deux méthodes de recherche, listez dans un tableau de 3 colonnes:
   - les noeuds de l'arbre
   - la valeur de la fonction d'évaluation
   - leur parent
   dans l'ordre dans lequel ils seront développés

Voici la table que je propose:

_Greedy Best First Search_
| Node | f(v) = h(v) | Parent |
|:-----:|:----:|:------:|
| S | 10 | None |
| F | 4 | S |
| G | 2 | F |

$A^*$
| Node | f(v) = g(v) + h(v) | Parent |
|:----:|:-----------:|:------:|
| S | 10 | None |
| A | 7 | S |
| C | 10 | A |
| F | 11 | S |
| G | 11 | C |


## Satisfaction de contraintes

Un architecte doit construire une maison de 4 pièces (1,2,3,4) organisées de la façon suivante:
- Le coté gauche de la maison est composé des pièces 1 et 2, le coté droit est composé des
chambres 3 et 4.
- Deux pièces sont adjacentes si elles partagent un mur complet (par exemple la pièce 2 est
adjacente aux pièces 1 et 3 mais pas à la pièce 4).

Vous devez décider où placer la cave C (l’une des quatre pièces), une fenêtre F et une porte
extérieure P. Les futurs propriétaires de la maison ont donné les contraintes suivantes:
- F ne peut pas être dans C, ni dans l’une des pièces adjacentes à C
- C et P ne peuvent pas être du même coté (droit ou gauche) de la maison
- La pièce contenant F doit être adjacente à au moins deux pièces
- La pièce contenant P doit être adjacente à au moins deux pièces


### Formulation du problème sous forme d'un CSP

1. Quelles sont les variables $x_i$ du problème? Est-ce que ce choix est unique ?

    _S’il y a plusieurs representations possibles, séléctionnez pour le reste de l’exercice celle qui possède le plus petit nombre de variables._

    a. On pourrait avoir comme variable de notre problème $x_1, x_2, x_3$ et $x_4$ avec $x_i$    représentant la pièce $i$. Ces variables pourraient alors prendre une valeur comprise dans $\{F, C, P\}$, qui représentent la cave, la fenêtre et la porte extérieure.

    b. On pourrait également avoir comme variable de notre problème $x_1, x_2$ et $x_3$ avec $x_1$ représentant la cave $C$, $x_2$ représentant la fenêtre $F$ et $x_3$ représentant la porte $P$. Les valeurs possibles de ces variables seraient alors une valeur prise dans $\{1, 2, 3, 4\}$, qui représentera le numéro de la pièce comme sur la figure de l'énoncé, indiquant dans quelle pièce se trouverait la cave la fenêtre ou la porte.

    Pour la suite de l'exercice, on considèrera les variables définies dans le 1.b.

2. Quelles sont les domaines $D_i$ des valeurs possibles $v_i$ pour chaque variable $x_i$ (ici on ne tient pas compte des contraintes) ?

    Les domaines $D_i$ des valeurs possibles $v_i$ pour chaque variable $x_i$ sont:
    $D_i = \{1, 2, 3, 4\}$ pour $i \in \{1, 2, 3\}$ si on ne considère pas les contraintes, 
    car sans contrainte, la Cave, la Porte extérieur et la Fenêtre peuvent se retrouver dans
    n'importe laquelle des pièces.

3. Quel est l'ensemble des contraintes $c_k$ sur les variables ?
   Représentez chacune des contraintes $c_k$ sous la forme:
   $$c_k = < x_1, \dots, x_n>:<v_11, \dots, v_1k>, <v_21, \dots, v_2m>$$
   
   Où $< x_1, · · · , x_n > est le tuple de $n$ variables relatives à la contrainte $c_k$ et
   $v_ij$ est l’ensemble des valeurs possibles du domaine $D_i$ de $x_i$ satisfaisant la contrainte.
   Par exemple:
   $$c_1 =< x_1, x_2 >: < 2, 4 >, < 1, 2 >$$
   represente une contrainte c1 binaire entre les deux variables x1 et x2. Cette contrainte
   nécessite que $x_1$ et $x_2$ prennent les valeurs 2 ou 4 et 1 ou 2, respectivement.
   
   L'ensemble des contraintes $c_k$ sur les variables $x_1$, $x_2$ et $x_3$ est donné par:
   (Rappel: $x_1$ = numéro de la pièce où se trouve la cave, $x_2$ = numéro de la pièce où se trouve la fenêtre et $x_3$ = numéro de la pièce où se trouve la porte extérieur.
   $$c_1 = <x_1, x_2>: <1, 3>, <1, 4>, <2, 4>, <3, 1>, <4, 1>, <4, 2>$$
   $$c_2 = <x_1, x_3>: <1, 3>, <1, 4>, <2, 3>, <2, 4>, <3, 1>, <3, 2>, <4, 1>, <4, 2>$$
   $$c_3 = <x_2>: <2>, <3>$$
   $$c_4 = <x_3>: <2>, <3>$$
   Avec $c_1$ la contrainte que la fenêtre ne peut pas être dans la cave, ni dans l'une des pièces adjacentes à la cave, $c_2$ la contrainte que la cave et la porte extérieure ne peuvent pas être du même côté, $c_3$ la contrainte que la pièce contenant la fenêtre doit être adjacente à au moins deux pièces, et $c_4$ la contrainte que la pièce contenant la porte extérieure doit être adjacente à au moins deux pièces.
        

### Backtracking

1. Utilisez la recherche backtracking avec forward checking pour touver un assignement de 
   valeurs aux variables qui satisfait toute les contraintes:
   
   Commencez par résoudre les contraintes unitaires. Vous devrez alternativement sélectionner une valeur pour une variable et appliquer le forward checking.

    Assignez les valeurs aux variables dans un ordre alphabétique/numérique sans utiliser aucune heuristique.

    Après chaque étape d’assignation et chaque étape de forward checking, indiquez pour chaque variable sa valeur assignée ou le domaine possible d’assignation.
    
    | $x_1 = C$ | $x_2 = F$ | $x_3 = P$ |
    |:---------:|:---------:|:---------:|
    | 1, 2, 3, 4 | 2, 3 | 2, 3 |
    | 1, 2,4 | __2__ | 2, 3 |
    | 4 | 2 | __2__ |
    | __4__ | 2 | 2 |

In [14]:
from copy import deepcopy

# This function indicate us if we find a solution.
# In fact, we just verify for our problem that all the variables are assignated, so
# that our dict has a len of 3.
def stop_condition(current: dict[str, int]) -> bool:
    if len(current) == 3:
        return True
    return False

# This function define the constraints of our problem
def define_constraints() -> dict[tuple[str, str], list[tuple[int, int]]]:
    return {
        ('C', 'F'): [(1, 3), (1, 4), (2, 4), (3, 1), (4, 1), (4, 2)],
        ('C', 'P'): [(1, 3), (1, 4), (2, 3), (2, 4), (3, 1), (3, 2), (4, 1), (4, 2)]
    }
     
# This function is responsible of the update of the domain of each variables for an assignation given.
def forward_checking(current: dict[str, int], domain: dict[str, list[int]], assign: tuple[str, int]) -> dict[str, list[int]]:
    newdomain = deepcopy(domain)
    constraints = define_constraints()
    tmp = []
    tmp2 = []

    # We iterate throught our domain and throught our constraints and if we have the element
    # assignated and the element of our domain in a constraint, we update the domain of the
    # element of the domain by creating a new list for this domain's variable (with no doublons)
    for item1 in newdomain:
        for (item2, item3) in constraints:
            if assign[0] == item2 or assign[0] == item3:
                if item1 == item2 or item1 == item3:
                    for (i, j) in constraints[(item2, item3)]:
                        if i == assign[1]:
                            tmp.append(j)
                    for i in newdomain[item1]:
                        if i in tmp:
                            tmp2.append(i)
                    newdomain[item1] = list(set(tmp2))
    return newdomain

# This function is the backtracking with forward checking. In current, we have the current assignation,
# and in domain, we have all the domain of the variables not yet assignated.
def backtracking(current: dict[str, int], domain: dict[str, list[int]]):
    # If we find a solution, we return it
    if stop_condition(current):
        return current
    # We print the current assignation and the current domain of non assignated variables
    print("Assignated variables: " + str(current))
    print("Domain of other variables: " + str(domain))
    value = list(domain.keys())[0]
    possibilities = domain.pop(value)
    # We try all the assignations of the first variable of our domain thanks to a recursive call to our function
    for i in possibilities:
        current[value] = i
        domain = forward_checking(current, domain, (value, i))
        for i in domain:
            # If our new domain has some variables with an empty domain, we return {} for an error
            if len(domain[i]) == 0:
                return {}
        return backtracking(current, domain)

domain = {'C': [1, 2, 3, 4], 'F': [2, 3], 'P': [2, 3]}
print("Result of the assignations by the backtracking: " + str(backtracking({}, domain)))


Assignated variables: {}
Domain of other variables: {'C': [1, 2, 3, 4], 'F': [2, 3], 'P': [2, 3]}
Assignated variables: {'C': 1}
Domain of other variables: {'F': [3], 'P': [3]}
Assignated variables: {'C': 1, 'F': 3}
Domain of other variables: {'P': [3]}
Result of the assignations by the backtracking: {'C': 1, 'F': 3, 'P': 3}
